# Подготовка данных

Импотируем необходимые библиотеки

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from scipy.stats import gaussian_kde

Читаем данные и конкатенируем `train_data` и `test_data` в один `data_frame`

In [3]:
train_data = pd.read_csv("titanic_data/train.csv")
test_data = pd.read_csv("titanic_data/test.csv")

data_frame = pd.concat([train_data, test_data], axis=0, ignore_index=True)

Посмотрим на типы всех столбцов

In [4]:
data_frame.dtypes

,0
PassengerId,int64
Survived,float64
Pclass,int64
Name,object
Sex,object
Age,float64
SibSp,int64
Parch,int64
Ticket,object
Fare,float64


Преобразуем столбы `Survived`, `Pclass`, `Sex`, `Embarked` в категориальный тип

In [5]:
for col in ['Survived', 'Pclass', 'Sex', 'Embarked']:
    data_frame[col] = data_frame[col].astype('category')
data_frame.dtypes

,0
PassengerId,int64
Survived,category
Pclass,category
Name,object
Sex,category
Age,float64
SibSp,int64
Parch,int64
Ticket,object
Fare,float64


Избавимся от дубликатов (если такие есть)

In [6]:
data_frame = data_frame.drop_duplicates(subset='PassengerId', keep='first')

Выведем итоговую таблицу, с которой будем работать

In [7]:
data_frame.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Анализ таблицы

Выведим общую статистику таблицы

In [8]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  1309 non-null   int64   
 1   Survived     891 non-null    category
 2   Pclass       1309 non-null   category
 3   Name         1309 non-null   object  
 4   Sex          1309 non-null   category
 5   Age          1046 non-null   float64 
 6   SibSp        1309 non-null   int64   
 7   Parch        1309 non-null   int64   
 8   Ticket       1309 non-null   object  
 9   Fare         1308 non-null   float64 
 10  Cabin        295 non-null    object  
 11  Embarked     1307 non-null   category
dtypes: category(4), float64(2), int64(3), object(3)
memory usage: 87.6+ KB


Выведим основные статистические характеристики

In [9]:
data_frame.describe()

,PassengerId,Age,SibSp,Parch,Fare
count,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,655.000000,29.881138,0.498854,0.385027,33.295479
std,378.020061,14.413493,1.041658,0.865560,51.758668
min,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,21.000000,0.000000,0.000000,7.895800
50%,655.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,39.000000,1.000000,0.000000,31.275000
max,1309.000000,80.000000,8.000000,9.000000,512.329200


Визуализируем часть статистики

Посмотрим на распределение мужчин и женщин по классам

In [10]:
grouped = data_frame.groupby(['Pclass', 'Sex'], observed=False).size().unstack()

grouped['total'] = grouped.sum(axis=1)

grouped = grouped[['male', 'female', 'total']]

print(grouped)

Sex     male  female  total
Pclass                     
1        179     144    323
2        171     106    277
3        493     216    709


Как можно увидеть, больше всего пассажиров было в 3-м классе. Также можно увидеть, что в каждом классе женщин меньше, чем мужчин (ярче всего эта разница видна в третьем классе). Построим Bar chart распеделения

In [11]:
plot_data = grouped.reset_index().melt(
    id_vars=['Pclass'],
    value_vars=['male', 'female'],
    var_name='Sex',
    value_name='Count'
)

fig = px.bar(
    plot_data,
    x='Pclass',
    y='Count',
    color='Sex',
    barmode='stack',
    title='Distribution of passengers by class and sex',
    color_discrete_map={'female': 'pink', 'male': 'lightblue'}
)

fig.update_layout(
    xaxis=dict(
        dtick=1,
        tickmode='linear'
    )
)

fig.show()

Сгруппируем таблицу в два уровня: класс и пол, по среднему значению возраста

In [12]:
group_df = data_frame.groupby(['Sex', 'Pclass'], observed=False)['Age'].mean()
group_df.sort_values()

,,Age
Sex,Pclass,
female,3,22.185329
male,3,25.962264
female,2,27.499223
male,2,30.815380
female,1,37.037594
male,1,41.029272


- Самые юные - женщины из 3-го класса
- Самые возростные - мужчины из 1-го класса

Найдем разницу между крайними значениями

In [13]:
group_df.max() - group_df.min()

18.843942575810384

Визуализируем распределение возрастов с помощью Box chart

In [14]:
fig = px.box(
    data_frame,
    x='Pclass',
    y='Age',
    color='Sex',
    color_discrete_map={'male': 'lightblue', 'female': 'pink'},
    title='Distribution of passengers by age and sex',
    hover_data=['Age']
)

fig.show()

Отберем выживших пассажиров с фамилией, начинающейся на “K” и отсортируем их по убыванию стоимости билета.

In [15]:
k_survive = data_frame[(data_frame['Survived'] == 1) & (data_frame['Name'].str.startswith('K'))].sort_values(by='Fare', ascending=False)
k_survive

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
621,622,1.0,1,"Kimball, Mr. Edwin Nelson Jr",male,42.0,1,0,11753,52.5542,D19,S
457,458,1.0,1,"Kenyon, Mrs. Frederick R (Marion)",female,NaN,1,0,17464,51.8625,D21,S
316,317,1.0,2,"Kantor, Mrs. Sinai (Miriam Sternin)",female,24.0,1,0,244367,26.0000,NaN,S
184,185,1.0,3,"Kink-Heilmann, Miss. Luise Gretchen",female,4.0,0,2,315153,22.0250,NaN,S
706,707,1.0,2,"Kelly, Mrs. Florence ""Fannie""",female,45.0,0,0,223596,13.5000,NaN,S
691,692,1.0,3,"Karun, Miss. Manca",female,4.0,0,1,349256,13.4167,NaN,C
303,304,1.0,2,"Keane, Miss. Nora A",female,NaN,0,0,226593,12.3500,E101,Q
300,301,1.0,3,"Kelly, Miss. Anna Katherine ""Annie Kate""",female,NaN,0,0,9234,7.7500,NaN,Q
573,574,1.0,3,"Kelly, Miss. Mary",female,NaN,0,0,14312,7.7500,NaN,Q


Среди них, больше всего заплатил **Kimball, Mr. Edwin Nelson Jr**

In [16]:
price = k_survive['Fare'].max()
k_survive[k_survive['Fare'] == price]['Name']

,Name
621,"Kimball, Mr. Edwin Nelson Jr"


А меньше всего - **Kelly, Miss. Anna Katherine "Annie Kate** и **Kelly, Miss. Mary**

In [17]:
price = k_survive['Fare'].min()
k_survive[k_survive['Fare'] == price]['Name']

,Name
300,"Kelly, Miss. Anna Katherine ""Annie Kate"""
573,"Kelly, Miss. Mary"


Найдем какое максимальное количество родных было с выжившим пассажиром

In [18]:
survived_data = data_frame[data_frame['Survived'] == 1]
(survived_data['SibSp'] + survived_data['Parch']).max()

6

Посчитаем среднюю стоимость билета пассажиров, для которых указана каюта (`Cabin`) и для тех, у кого она не указана

In [19]:
fare_means = data_frame.groupby(data_frame['Cabin'].notna())['Fare'].mean()
mean_fare_with = fare_means[True]
mean_fare_without = fare_means[False]
mean_fare_with, mean_fare_without

(np.float64(81.92899830508475), np.float64(19.132707206317868))

Во сколько раз они отличаются?

In [20]:
mean_fare_with / mean_fare_without

np.float64(4.282143526350036)

Построим **Scatter plot** для отображения распределения пассажиров по возрасту и стоймости билета, включая пол

In [33]:
filtered_data = data_frame.dropna(subset=['Age', 'Fare', 'Sex'])

fig = px.scatter(
    filtered_data,
    x='Age',
    y='Fare',
    color='Sex',
    color_discrete_map={'female': 'pink', 'male': 'lightblue'},
    title='Chart of age and ticket prices by sex',
    hover_data=['Pclass']
)

fig.update_layout(
    plot_bgcolor='white'
)

fig.show()

 Построеим **Histogram** возрастного распределения пассажиров из Саутгемптона

In [34]:
southampton_passengers = data_frame[
    (data_frame['Embarked'] == 'S') &
    (data_frame['Age'].notna())
]

fig = px.histogram(
    southampton_passengers,
    x='Age',
    nbins=40,
    title='Age distribution of passengers from Southampton',
    color_discrete_sequence=['pink'],
    opacity=0.7,
    hover_data=['Sex', 'Pclass']
)

fig.update_traces(marker_line_width=0, selector=dict(type='histogram'))
fig.update_layout(bargap=0.05)

ages = southampton_passengers['Age'].values
kde = gaussian_kde(ages)
x_grid = np.linspace(ages.min(), ages.max(), 100)
y_kde = kde(x_grid) * len(ages) * (ages.max() - ages.min()) / 30

fig.add_trace(
    go.Scatter(
        x=x_grid,
        y=y_kde,
        mode='lines',
        line=dict(color='red', width=2),
        name='Distribution density',
        hoverinfo='skip'
    )
)

fig.update_layout(
    xaxis_title='Age',
    yaxis_title='Passenger count',
    hovermode='x',
    showlegend=True,
    height=500
)

fig.show()

Построим **Linear plot**, показывающая процент выживших по возрастным группам с разделением на пол

In [40]:
temp_df = data_frame[['Age', 'Sex', 'Survived']].dropna().copy()

age_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80]
age_labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70+']

temp_df['Survived'] = temp_df['Survived'].astype(float)

fig = px.line(
    temp_df.assign(
        AgeGroup=lambda x: pd.cut(x['Age'], bins=age_bins, labels=age_labels)
    ).groupby(['AgeGroup', 'Sex'], observed=True)['Survived'].mean().mul(100).reset_index(name='SurvivalRate'),

    x='AgeGroup',
    y='SurvivalRate',
    color='Sex',
    color_discrete_map={'female': 'pink', 'male': 'lightblue'},
    title='Percentage of survivors by age group',
    markers=True,
    line_shape='linear'
)

fig.update_layout(
    xaxis_title='Age',
    yaxis_title='Percentage of survivors (%)',
    plot_bgcolor='white',
    hovermode='x unified',
    yaxis=dict(range=[0, 100]),
    legend_title_text='Sex'
)

fig.show()


Построим **Horizontal bar chart** для отображения средней стоимости билета по классам на Титанике

In [35]:
fare_by_class = data_frame.groupby('Pclass', observed=True)['Fare'].mean().reset_index()

fare_by_class['Class'] = fare_by_class['Pclass'].map({
    1: '1st class',
    2: '2nd class',
    3: '3rt class'
})

fig = px.bar(
    fare_by_class,
    y='Class',
    x='Fare',
    orientation='h',
    title='Average ticket price by class on the Titanic',
    color='Class',
    color_discrete_map={
        '1st class': 'yellow',
        '2nd class': 'yellowgreen',
        '3rt class': 'green'
    },
    text_auto='.2f',
    height=400
)

fig.update_layout(
    xaxis_title='Average cost',
    yaxis_title='',
    plot_bgcolor='white',
    showlegend=False,
    hovermode='y unified',
    margin=dict(l=100, r=50, t=80, b=50)
)

fig.update_traces(
    textposition='inside',
    insidetextanchor='middle',
    hovertemplate='<b>%{y}</b><br>Average cost: %{x:.2f}<extra></extra>',
    marker_line=dict(width=1, color='DarkSlateGray')
)

fig.show()

Построим **Pie chart**, на котором отобразим порты отправления пассажиров

In [36]:
embarked_counts = data_frame['Embarked'].value_counts().reset_index()
embarked_counts.columns = ['Embarked', 'Count']

embarked_counts['Embarked'] = embarked_counts['Embarked'].map({
    'C': 'Cherbourg',
    'Q': 'Queenstown',
    'S': 'Southampton'
})

fig = px.pie(
    embarked_counts,
    values='Count',
    names='Embarked',
    title='Port proportions',
    color='Embarked',
    color_discrete_map={'Cherbourg': 'lightblue', 'Queenstown': 'lightcoral', 'Southampton': 'lightgreen'}
    )
fig.show()

Построим **Sunburst chart**, на котором отобразим информацию о статусе, поле и классе пассажиров

In [37]:
data_sunburst = data_frame[['Survived', 'Sex', 'Pclass']].dropna(subset=['Survived'])
data_sunburst['Survived'] = data_sunburst['Survived'].map({0: 'Not Survive', 1: 'Survived'})

fig = px.sunburst(
    data_sunburst,
    path=['Survived', 'Sex', 'Pclass'],
    color='Survived',
    color_discrete_map={'Not Survive': 'lightcoral', 'Survived': 'lightgreen'},
    title='Survival → Sex → Pclass Distribution'
)

fig.show()

/usr/local/lib/python3.11/dist-packages/plotly/express/_core.py:1727: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/usr/local/lib/python3.11/dist-packages/plotly/express/_core.py:1727: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/usr/local/lib/python3.11/dist-packages/plotly/express/_core.py:1727: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Сделаем **Sankey Diagram** распределение пассажиров Титаника между портами и классами

In [38]:
sankey_data = data_frame.groupby(['Embarked', 'Pclass'], observed=False).size().reset_index(name='Count')

labels = [
    'Cherbourg (C)', 'Queenstown (Q)', 'Southampton (S)',
    '1st Class', '2nd Class', '3rd Class'
]

node_indices = {
    'C': 0, 'Q': 1, 'S': 2,
    1: 3, 2: 4, 3: 5
}

sources = []
targets = []
values = []
link_colors = []

port_colors = {
    'C': 'rgba(173, 216, 230, 0.8)',
    'Q': 'rgba(240, 128, 128, 0.8)',
    'S': 'rgba(144, 238, 144, 0.8)'
}

for _, row in sankey_data.iterrows():
    sources.append(node_indices[row['Embarked']])
    targets.append(node_indices[row['Pclass']])
    values.append(row['Count'])
    link_colors.append(port_colors[row['Embarked']])

fig = go.Figure(go.Sankey(
    node=dict(
        pad=20,
        thickness=30,
        line=dict(color='black', width=0.5),
        label=labels
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color=link_colors,
        hoverinfo='all',
        hovertemplate='%{source.label} → %{target.label}<br>Passengers: %{value}<extra></extra>'
    )
))

fig.update_layout(
    title_text='Departure port → Cabin class',
    font=dict(size=12, family='Arial'),
    hovermode='x',
    annotations=[
        dict(
            x=0.5, y=-0.1,
            xref='paper', yref='paper',
            text='',
            showarrow=False
        )
    ]
)

fig.show()

Построим 3d-график, отображающий возраст, стоимость билета и класс для каждого пассажира

In [39]:
fig = px.scatter_3d(
    data_frame,
    x='Age',
    y='Fare',
    z='Pclass',
    color='Sex',
    color_discrete_map={'female': 'pink', 'male': 'lightblue'},
    title='3D Scatter: Age, Fare, and Pclass',
    hover_data=['Name', 'Sex', 'Age', 'Pclass', 'Fare', 'Embarked', 'Survived']
)

fig.show()